# Deeper Dive Into Optimization Techniques

## Option 1 : Avoid Shuffling and Minimize Data Movement
So, what's the alternatives? Here it is:

- 1.1 Use `Broadcast Joins` for small tables - when joining a small DataFrame with a large DataFrame, always use broadcast joins to avoid expensive shuffling
- 1.2 Use `Bucketing` instead of Shuffling for Joins

### 1.1 Using Broadcast Joins Instead of Shuffling for Joins


In [12]:
# Using Broadcast joins for Small Tables
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast
import time

spark = SparkSession.builder.appName('BroadcastJoinExample').getOrCreate()

df_large = spark.read.csv('./resources/8_large_file.csv', header=True, inferSchema=True)
df_small = spark.read.csv('./resources/9_small_file.csv', header=True, inferSchema=True)

start_time = time.time()
df_joined = df_large.join(broadcast(df_small), 'id')
df_joined.count()
end_time = time.time()
print(f"Execution Time (With Broadcast): {end_time - start_time} seconds")



# without broadcasting
df_large1 = spark.read.csv('./resources/8_large_file.csv', header=True, inferSchema=True)
df_small1 = spark.read.csv('./resources/9_small_file.csv', header=True, inferSchema=True)

start_time = time.time()
df_joined1 = df_large1.join(df_small1, 'id')
df_joined1.count()
end_time = time.time()
print(f"Execution Time (Without Broadcast): {end_time - start_time} seconds")


# use .explain() to view query plans
print('----------------NOW WITH NO BROADCAST (Look for (strategy=broadcast))--------------')
df_joined.explain(True)
print('=================================================================================')
print('----------------NOW WITH NO BROADCAST (Look for Shuffle Operations)--------------')
df_joined1.explain(True)

spark.stop()

25/03/29 21:45:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/29 21:45:54 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/03/29 21:45:54 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/03/29 21:45:54 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/03/29 21:45:54 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


Execution Time (With Broadcast): 0.1299881935119629 seconds
Execution Time (Without Broadcast): 0.13207006454467773 seconds
----------------NOW WITH NO BROADCAST (Look for (strategy=broadcast))--------------
== Parsed Logical Plan ==
'Join UsingJoin(Inner, [id])
:- Relation [id#957,name#958,salary#959,department#960] csv
+- ResolvedHint (strategy=broadcast)
   +- Relation [id#982,bonus_percentage#983] csv

== Analyzed Logical Plan ==
id: int, name: string, salary: int, department: string, bonus_percentage: double
Project [id#957, name#958, salary#959, department#960, bonus_percentage#983]
+- Join Inner, (id#957 = id#982)
   :- Relation [id#957,name#958,salary#959,department#960] csv
   +- ResolvedHint (strategy=broadcast)
      +- Relation [id#982,bonus_percentage#983] csv

== Optimized Logical Plan ==
Project [id#957, name#958, salary#959, department#960, bonus_percentage#983]
+- Join Inner, (id#957 = id#982), rightHint=(strategy=broadcast)
   :- Filter isnotnull(id#957)
   :  +- Rela

### 1.2 Using Bucketing Instead of Shuffling for Joins
